In [75]:
import cv2 as cv
import numpy as np
import argparse
import imageio
import matplotlib.pyplot as plt
%matplotlib widget

W = 20          # window size is WxW
C_Thr = 0.43    # threshold for coherency
LowThr = 35     # threshold1 for orientation, it ranges from 0 to 180
HighThr = 57    # threshold2 for orientation, it ranges from 0 to 180

def calcGST(inputIMG, w):
    img = inputIMG.astype(np.float32)
    # GST components calculation (start)
    # J =  (J11 J12; J12 J22) - GST
    imgDiffX = cv.Sobel(img, cv.CV_32F, 1, 0, 3)
    imgDiffY = cv.Sobel(img, cv.CV_32F, 0, 1, 3)
    imgDiffXY = cv.multiply(imgDiffX, imgDiffY)
    
    imgDiffXX = cv.multiply(imgDiffX, imgDiffX)
    imgDiffYY = cv.multiply(imgDiffY, imgDiffY)
    J11 = cv.boxFilter(imgDiffXX, cv.CV_32F, (w,w))
    J22 = cv.boxFilter(imgDiffYY, cv.CV_32F, (w,w))
    J12 = cv.boxFilter(imgDiffXY, cv.CV_32F, (w,w))
    # GST components calculations (stop)
    # eigenvalue calculation (start)
    # lambda1 = 0.5*(J11 + J22 + sqrt((J11-J22)^2 + 4*J12^2))
    # lambda2 = 0.5*(J11 + J22 - sqrt((J11-J22)^2 + 4*J12^2))
    tmp1 = J11 + J22
    tmp2 = J11 - J22
    tmp2 = cv.multiply(tmp2, tmp2)
    tmp3 = cv.multiply(J12, J12)
    tmp4 = np.sqrt(tmp2 + 4.0 * tmp3)
    lambda1 = 0.5*(tmp1 + tmp4)    # biggest eigenvalue
    lambda2 = 0.5*(tmp1 - tmp4)    # smallest eigenvalue
    # eigenvalue calculation (stop)
    # Coherency calculation (start)
    # Coherency = (lambda1 - lambda2)/(lambda1 + lambda2)) - measure of anisotropism
    # Coherency is anisotropy degree (consistency of local orientation)
    imgCoherencyOut = cv.divide(lambda1 - lambda2, lambda1 + lambda2)
    # Coherency calculation (stop)
    # orientation angle calculation (start)
    # tan(2*Alpha) = 2*J12/(J22 - J11)
    # Alpha = 0.5 atan2(2*J12/(J22 - J11))
    imgOrientationOut = cv.phase(J22 - J11, 2.0 * J12, angleInDegrees = True)
    imgOrientationOut = 0.5 * imgOrientationOut
    # orientation angle calculation (stop)
    return imgCoherencyOut, imgOrientationOut
# path = '211223_ExNo_93_60x_FM4-64+trans_03_2000-4180_trans_kymo_forCoco.tif'
# path = 'kymograph3.tif'
# path = 'kymograph5.tif'
# path = 'kymograph_5 filtered_backward.tif'
# path = 'kymograph_5 filtered_forward.tif'
path = "kymograph_1/kymograph_1 filtered_forward.tif"
# path = "kymograph_1/kymograph1.tif"

# path = '211223_ExNo_93_60x_FM4-64+trans_03_2000-4180_trans_kymo.png'
imgIn = imageio.imread(path)
imgcrop = (imgIn[:300,:])
# imgcrop = cv2.cvtColor(imgcrop,cv2.COLOR_BGR2GRAY)   

# imgIn = cv.imread(path, cv.IMREAD_GRAYSCALE)
# imgcrop = (imgIn[:300,:]).astype(int)*4

im_blurred =cv.GaussianBlur(imgcrop, (51, 51),0)
imgcrop = (imgcrop)/((im_blurred==0)*np.ones(im_blurred.shape)+im_blurred)*120
imgCoherency, imgOrientation = calcGST(imgcrop, W)
, imgCoherencyBin = cv.threshold(imgCoherency, C_Thr, 255, cv.THRESH_BINARY)
_, imgOrientationBin = cv.threshold(imgOrientation, LowThr, HighThr, cv.THRESH_BINARY)

# # imgBin = cv.bitwise_and(imgCoherencyBin, imgOrientationBin)



In [64]:
imgOrientationBin

array([[57., 57., 57., ..., 57., 57., 57.],
       [57., 57., 57., ..., 57., 57., 57.],
       [57., 57., 57., ..., 57., 57., 57.],
       ...,
       [57., 57., 57., ..., 57., 57., 57.],
       [57., 57., 57., ..., 57., 57., 57.],
       [57., 57., 57., ..., 57., 57., 57.]], dtype=float32)

In [60]:
fig, ax = plt.subplots()
ax.imshow(im_blurred,cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
fig, ax = plt.subplots()
ax.imshow(imgIn[0:300,:],cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
fig, ax = plt.subplots()
ax.imshow(imgcrop,cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
fig, ax = plt.subplots()
ax.imshow(imgCoherency,cmap="gray")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
fig, ax = plt.subplots()
ax.imshow(imgcrop,cmap="gray")
ax2 = ax.twiny()
ax2.plot(np.tan((np.mean(imgOrientation,axis=1)-90)/180*np.pi),range(len(np.mean(imgOrientation,axis=1))),color = "red")
# ax.imshow(imgOrientation,alpha=0.3,cmap="jet")
# ax.imshow(0*imgIn,cmap='jet',alpha = 0.3)
ax.imshow(imgOrientation*(imgCoherency>0.3),alpha=0.5,cmap="jet")
# ax2.set_xlim((-4,4))
# ax2.set_xlabel('speed')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
fig, ax = plt.subplots()
ax.imshow(imgIn,cmap="gray")
ax2 = ax.twiny()
# ax2.plot(np.tan((np.mean(imgOrientation,axis=1)-90)/180*np.pi),range(len(np.mean(imgOrientation,axis=1))),color = "red")
# ax.imshow(imgOrientation,alpha=0.3,cmap="jet")
# ax.imshow(0*imgIn,cmap='jet',alpha = 0.3)
# ax.imshow(imgOrientation*(imgCoherency>0.4),alpha=0.5,cmap="jet")
ax2.set_xlim((-4,4))
ax2.set_xlabel('speed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'speed')

In [31]:
import pandas as pd
line = np.tan((np.mean(imgOrientation,axis=1)-90)/180*np.pi)
data = pd.read_csv("211220_ExNo_90_60x_FM4-64+trans_01_460-2100_df_total.csv")

In [37]:
len(data["Molecule#"].unique())

59

In [54]:
data.loc[data['frame']<300]["Molecule#"].unique()

array([ 9011, 90122,  9014,  9016,  9019], dtype=int64)

In [59]:
mol = data.loc[data["Molecule#"]==9011]

In [45]:
mol

,timepoint,frame,time[s],x_position[nm],y_position[nm],x_displacement[nm],y_displacement[nm],Euclidean_p2p_distance[nm],Speed[um/s],Velocity[um/s],Speed_rolling_average[um/s],Velocity_rolling_average[um/s],Alpha,Gs,MSD,Dataset,Molecule#
3256,0,116.0,5.75,1239.792725,14589.869141,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,1.816119,<map object at 0x000002B328BD8C40>,NaN,NaN,9019
3257,1,117.0,5.80,1387.794922,14376.097656,-148.002197,213.771485,260.005573,5.200111,5.200111,NaN,NaN,1.862475,<map object at 0x000002B328BD8C40>,5.200111,NaN,9019
3258,2,118.0,5.85,1605.792480,14220.036133,-217.997558,156.061523,268.100978,5.362020,5.362020,NaN,NaN,1.795576,<map object at 0x000002B328BD8C40>,5.362020,NaN,9019
3259,3,119.0,5.90,1700.908081,14087.122070,-95.115601,132.914063,163.441505,3.268830,3.268830,NaN,NaN,1.697428,<map object at 0x000002B328BD8C40>,3.268830,NaN,9019
3260,4,120.0,5.95,1789.363525,14006.610352,-88.455444,80.511718,119.609792,2.392196,2.392196,NaN,NaN,1.687453,<map object at 0x000002B328BD8C40>,2.392196,NaN,9019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3348,92,208.0,10.35,14433.859375,7835.968750,338.160156,-139.693848,365.877934,7.317559,-7.317559,4.286188,1.111992,NaN,<map object at 0x000002B328BD8C40>,7.317559,NaN,9019
3349,93,209.0,10.40,12465.467773,7815.238770,1968.391602,20.729980,1968.500757,39.370015,39.370015,7.943389,4.769193,NaN,<map object at 0x000002B328BD8C40>,39.370015,NaN,9019
3350,94,210.0,10.45,11592.162109,8004.518066,873.305664,-189.279296,893.582360,17.871647,-17.871647,9.357546,2.609021,NaN,<map object at 0x000002B328BD8C40>,17.871647,NaN,9019
3351,95,211.0,10.50,10431.039063,8092.572266,1161.123046,-88.054200,1164.457071,23.289141,-23.289141,11.237090,-0.169264,NaN,<map object at 0x000002B328BD8C40>,23.289141,NaN,9019


In [62]:
fig, ax = plt.subplots()

ax.scatter(mol['time[s]'],mol['Velocity_rolling_average[um/s]'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
fig, ax = plt.subplots()
ax.scatter(mol['time[s]'],mol['y_position[nm]'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
s1 = line[160:2178]
s2 = data["Vnet"]
s1 = (s1-np.mean(s1))/np.std(s1)
s2 = (s2-np.mean(s2))/np.std(s2)
sd = 1/np.sqrt(data["Vtot"])
lags = np.array(range(-len(s1)//2,len(s1)//2))*0.05

In [97]:
fig, ax = plt.subplots()
ax.plot(data["Vnet"])
ax.plot(data["Vnet"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.plot(data["frame"]*0.05,s1,label="bulk flow")
ax2.plot(data["frame"]*0.05,s2,color='red',label="straight trajectory fluorescence")
ax.set_xlabel("time(s)")
# ax.set_yabel("")
ax.legend(loc = "upper right")
ax2.legend(loc = "upper left")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
c = correlate(s1,s1,"same")
fig, ax = plt.subplots()
ax.plot(lags,c/len(s1),label="bulk flow")
c = correlate(s2,s2,"same")
ax.plot(lags,c/len(s1),label = "straight trajectory fluorescence",color = "red")
ax.set_xlabel("lag (s)")
ax.legend()
ax.plot(lags,np.zeros(len(lags))+2/np.sqrt(len(s1)/2),color="black",alpha = 0.3)
ax.plot(lags,np.zeros(len(lags))-2/np.sqrt(len(s1)/2),color="black",alpha = 0.3)
# ax2.legend(loc = "upper left")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:

from scipy.signal import correlate
c = correlate(s1,s2,"same","direct")
fig, ax = plt.subplots()
ax.plot(lags,c/len(s1))
ax.plot(lags,np.zeros(len(lags))+2/np.sqrt(len(s1)/2),color="black",alpha = 0.3)
ax.plot(lags,np.zeros(len(lags))-2/np.sqrt(len(s1)/2),color="black",alpha = 0.3)
ax.set_title('cross correlation')
ax.set_xlabel("lag (s)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'lag (s)')

In [94]:
from scipy.signal import correlate
s1 = np.sin(np.linspace(0,10,100))
s2 = np.sin(np.linspace(0,10,100)+1)

c = correlate(s1,s2,"same","direct")
fig, ax = plt.subplots()
ax.plot(np.linspace(-5,5,100),c/len(s1))
ax.set_title('cross correlation')
ax.set_xlabel("lag (s)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'lag (s)')

In [95]:
fig, ax = plt.subplots()
ax.plot(s1,label="bulk flow")
ax.plot(s2,color='red',label="straight trajectory fluorescence")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
np.save('speeds',np.tan((np.mean(imgOrientation,axis=1)-90)/180*np.pi))

In [66]:
fig, ax = plt.subplots()
ax.imshow(imgIn,cmap="gray")
ax.imshow(imgOrientation,alpha=0.3,cmap="jet")
# ax.imshow(0*imgIn,cmap='jet',alpha = 0.3)
# ax.imshow(imgOrientation*(imgCoherency>0.6)*(imgOrientation<75),alpha=0.5,cmap="jet")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
fig, ax = plt.subplots()
ax.plot(np.mean(imgOrientation,axis=1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
fig, ax = plt.subplots()
ax.imshow(imgIn,cmap="gray")
ax.imshow(imgCoherency,alpha=0.3,cmap="jet")
# ax.imshow(0*imgIn,cmap='jet',alpha = 0.3)
# ax.imshow(imgOrientation*(imgCoherency>0.6),alpha=0.5,cmap="jet")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fig, ax = plt.subplots()
ax.imshow(imgIn,cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …